In [1]:
using LinearAlgebra, ForwardDiff

In [2]:
kmax = 200
ϵ=10^(-6)

1.0e-6

In [14]:
function lagrange_newton(f,hs,x_0,μ_0)
    ∇f(x) = ForwardDiff.gradient(f, x)
    ∇2f(x) = ForwardDiff.jacobian(∇f, x)
    ∇hs = [x -> ForwardDiff.gradient(h, x) for h in hs]
    
    ∇h2s = [x -> ForwardDiff.jacobian(h, x) for h in ∇hs]
    ℒ(x,μs) = f(x) + sum(μ * h(x) for (μ,h) in zip(μs,hs))
    ∂ℒ∂x(x,μs) = ∇f(x) + sum(μ * ∇h(x) for (μ,∇h) in zip(μs,∇hs))
    ∂ℒ∂λ(x,μs) = [h(x) for h in hs]
    L_0 = ∂ℒ∂x(x_0,μ_0)
    h_0 = ∂ℒ∂λ(x_0,μ_0) 

    phi_0 = vcat(L_0,h_0)
    iter=0
    p =length(hs)
    n = length(x_0)
    X=[x_0]::Vector{Vector{Float64}}
    Mu=[μ_0]::Vector{Vector{Float64}}
    while norm(phi_0)>ϵ && iter<kmax
        iter+=1
        println(norm(phi_0))
        hessian = ∇2f(x_0)
        for i in 1:p
            hessian+=μ_0[i]*∇h2s[i](x_0)
        end
        for i in 1:p
            hessian = hcat(hessian,∇hs[i](x_0))
        end
        temp = ∇hs[1](x_0)'
        for i =2:p
            temp = vcat(temp,∇hs[i](x_0)')
        end
        temp = hcat(temp,zeros(p,p))
        hessian = vcat(hessian,temp)
        delta = -hessian\phi_0
        delta_x = delta[1:n]
        delta_μ = delta[n+1:n+p]
        x_0=x_0+delta_x
        μ_0=μ_0+delta_μ

        L_0 = ∂ℒ∂x(x_0,μ_0)
        h_0 = ∂ℒ∂λ(x_0,μ_0) 
        
        phi_0 = vcat(L_0,h_0)
        
        push!(X,x_0)
        push!(Mu,μ_0)

    end
    return X,Mu,iter
end

lagrange_newton (generic function with 1 method)

In [15]:
f(x) = 2*x[1]^4+x[2]^4+4*x[1]^2-x[1]*x[2]+6*x[2]^2
h(x) = 2*x[1]-x[2]+4

h (generic function with 1 method)

In [16]:
X,Mu,iter=lagrange_newton(f,[h],[0.,0.],[0.])
for i =1:iter+1
    println(X[i],' ',Mu[i])
end

4.0
44.305796340071076
5.155005942054521
0.014977986102174719
[0.0, 0.0] [0.0]
[-1.7692307692307692, 0.46153846153846156] [7.3076923076923075]
[-1.4523919988334946, 1.0952160023330109] [16.60806234685793]
[-1.4678436439052165, 1.0643127121895668] [19.049612958985136]
[-1.4679481134191408, 1.0641037731617182] [19.05680332151563]


In [6]:
hh = [8 -1 2;-1 12 -1;2 -1 0]

3×3 Matrix{Int64}:
  8  -1   2
 -1  12  -1
  2  -1   0

In [7]:
p = [0; 0; 4]

3-element Vector{Int64}:
 0
 0
 4

In [8]:
println(hh\p)

[1.7692307692307692, -0.46153846153846156, -7.3076923076923075]


In [9]:
g(x) = 1000 - x[1]^2-2*x[2]^2-x[3]^2-x[1]*x[2]-x[1]*x[3]
hh1(x) = x[1]^2+x[2]^2+x[3]^2-25
hh2(x)= 8*x[1]+14*x[2]+7*x[3]-56

hh2 (generic function with 1 method)

In [10]:
X2,Mu2,iter2=lagrange_newton(g,[hh1,hh2],[3,-.2,3],[0.,0.])
for i =1:iter2+1
    println(X2[i],' ',Mu2[i])
end

20.053967188563963
2.658234546461622
0.06180284051918647
0.0008120789740890623
[3.0, -0.2, 3.0] [0.0, 0.0]
[3.691272302368432, 0.1448429284077118, 3.491717226192082] [1.3773201204440253, 0.3443980045840636]
[3.5341892087710636, 0.2130278584864499, 3.5348709015744566] [1.226249628526347, 0.2745177511564703]
[3.512232995511023, 0.21692122660935237, 3.5521769804829835] [1.2234663964043355, 0.2749360404862345]
[3.5121213429077875, 0.21698794020251688, 3.5521711562717804] [1.2234635606459223, 0.274937101961764]
